<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Aula_5_Transforma%C3%A7%C3%B5es_com_RDDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("drive/My\ Drive/My\ Professional\ Carrer/Spark\ course/virtual_classroom/colab_test/test.csv") \
  .show()

AnalysisException: ignored

# Transformações com RDDs
Nessa aula veremos algumas das mais comuns transformações utilizanod RDDs. Lembrando que todas as transformações High-level realizadas em dataframes são convertidas em transformações low-level utilizando os RDDs.

In [32]:
sc = spark.sparkContext

### Coleção a ser utilizada
Durante essa parte do curso, veremos como aplicar os comandos utilizando a seguinte coleção em Python (texto extraído da seção Overwiew em http://spark.apache.org/docs/latest/rdd-programming-guide.html#overview

In [ ]:
text = "At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures. A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only added to, such as counters and sums"
words = text.split(" ")

### Criando o RDD

In [ ]:
# Look for the core number
rdd = sc.parallelize(words, 4)
rdd.setName("myWords")

myWords ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [ ]:
rdd.take(10)

['At',
 'a',
 'high',
 'level,',
 'every',
 'Spark',
 'application',
 'consists',
 'of',
 'a']

### Transformações
##### rdd.map()
Assumindo que temos:
```python
rdd2 = rdd1.map(<FUNCAO>)
```
A transformação `map()` irá mapear cada linha de `rdd1` em uma linha em `rdd2`

In [ ]:
''''
Cada linha do rdd irá ser mapeada para uma nova linha contendo 
a palavra, o tamanho da palavra e se começa com a letra 'S'
'''
def starts_with_S(word):
    return word.startswith("S")

new_rdd = rdd.map(lambda word: (word, len(word), starts_with_S(word)))

In [ ]:
new_rdd.take(20)

[('At', 2, False),
 ('a', 1, False),
 ('high', 4, False),
 ('level,', 6, False),
 ('every', 5, False),
 ('Spark', 5, True),
 ('application', 11, False),
 ('consists', 8, False),
 ('of', 2, False),
 ('a', 1, False),
 ('driver', 6, False),
 ('program', 7, False),
 ('that', 4, False),
 ('runs', 4, False),
 ('the', 3, False),
 ('user’s', 6, False),
 ('main', 4, False),
 ('function', 8, False),
 ('and', 3, False),
 ('executes', 8, False)]

##### rdd.filter()
Assumindo que temos:
```python
rdd2 = rdd1.filter(<FUNCAO>)
```
A transformação `filter()` irá remover linhas que resultem em `False` quando aplicadas à `<FUNCAO>`

In [ ]:
''''
Todas as linhas cuja posição 2 seja False serão removidas
'''
filtered_rdd = new_rdd.filter(lambda row: row[2]) # ou row[2] == True

In [ ]:
filtered_rdd.take(10)

[('Spark', 5, True),
 ('Spark', 5, True),
 ('Scala', 5, True),
 ('Spark', 5, True),
 ('Spark', 5, True),
 ('Spark', 5, True),
 ('Sometimes,', 10, True),
 ('Spark', 5, True)]

##### rdd.flatMap()
Assumindo que temos:
```python
rdd2 = rdd1.flatMap(<FUNCAO>)
```
A transformação `flatMap()` irá mapear cada linha de `rdd1` em uma ou mais linhas em `rdd2`

In [ ]:
list("igor")

['i', 'g', 'o', 'r']

In [ ]:
''''
Todas as palavras do rdd filtrado serão convertidas em um RDD contendo apenas
letras
'''

flatted_rdd = filtered_rdd.flatMap(lambda row: list(row[0]))

In [ ]:
flatted_rdd.take(20)

['S',
 'p',
 'a',
 'r',
 'k',
 'S',
 'p',
 'a',
 'r',
 'k',
 'S',
 'c',
 'a',
 'l',
 'a',
 'S',
 'p',
 'a',
 'r',
 'k']

##### rdd.distinct()
Assumindo que temos:
```python
rdd2 = rdd1.distinct()
```
A transformação `distinct()` irá gerar um novo rdd com apenas objetos distintos

In [ ]:
''''
Obteremos a lista de palavras únicas na frase
'''

distinct_rdd = rdd.distinct()

In [ ]:
distinct_rdd.take(10)

['At',
 'of',
 'operations',
 'provides',
 'in',
 'are',
 'other',
 'Hadoop-supported',
 'an',
 'may']

In [ ]:
distinct_rdd.count()
#rdd.count()

124

##### rdd.sortBy()
Assumindo que temos:
```python
rdd2 = rdd1.sortBy(<FUNCAO>)
```
A transformação `sortBy()` irá ordenar as linhas do rdd1 conforme critério definido na `<FUNCAO>`

In [ ]:
''''
Todas as palavras do rdd serão ordenadas conforme o tamanho da palavra
'''

sorted_distinct_rdd = distinct_rdd.sortBy(lambda word: len(word) * -1) # -1 significa order reversa

In [ ]:
sorted_distinct_rdd.take(20)

['Hadoop-supported',
 'accumulators,',
 'automatically',
 'transforming',
 'operations.',
 'partitioned',
 'application',
 'abstraction',
 'distributed',
 'efficiently',
 'operations',
 'variables,',
 'Sometimes,',
 'variables:',
 'collection',
 'failures.',
 'different',
 'broadcast',
 'resilient',
 'parallel.']

### Pair RDDs
Alguns RDDs específicos no Spark permitem a aplicação de transformações de modo mais eficiente. Estes são chamados de PairRDDs porque se apresentam na forma de um rdd key-value, ou seja, (K,V). As transformações disponíveis para serem aplicadas nos PairRDDs normalmente segue o padrão `<some-operation>ByKey`

##### Criando um PairRDD

In [ ]:
rdd

In [ ]:
pair_rdd = rdd.map(lambda x: (x, 1))
pair_rdd.take(10)

##### Criando um PairRDD utilizanod o keyBy
É uma transformação que cria um PairRDD passando-se uma função para retornar qual será a key

In [ ]:
key_by_pair_rdd = rdd.keyBy(lambda x: x[0].lower())

In [ ]:
key_by_pair_rdd.take(10)

##### rdd.mapValues()
Assumindo que temos:
```python
rdd2 = rdd1.mapValues(<FUNCAO>)
```
Transformação aplicada somente no valor de um PairRDD, aplicando uma `<FUNCAO>` e transformando cada linha do rdd1 em outra linha no rdd2

In [ ]:
letters = key_by_pair_rdd.mapValues(lambda x: 1)
letters.take(5)

##### rdd.reduceByKey
Realiza uma operação de reduce em um PairRDD baseado em seus valores agrupando por key

In [ ]:
# Realizando a contagem das palavras
reduced_pair_rdd = pair_rdd.reduceByKey(lambda a, b: a + b)
reduced_pair_rdd.take(10)

In [ ]:
# Realizando a contagem que palavras que começam com as letras do alfabeto
reduced_letters = letters.reduceByKey(lambda a, b: a + b)
reduced_letters.take(10)

##### rdd.sortByKey
Realiza uma operação de sort em um PairRDD baseado em sua key

In [ ]:
sorted_reduced_rdd = reduced_pair_rdd.sortByKey(ascending=False)
sorted_reduced_rdd.take(10)

In [ ]:
sorted_reduced_letters = reduced_letters.sortByKey(ascending=False)
sorted_reduced_letters.take(10)

##### rdd.join()
Transformação responsável por realizar a junção entre dois dataframes. Estando ambos os rdds na forma de um PairRDD, para executar o `.join()`, basta rodar o código abaixo:
```python
joinned_rdd = rdd1.join(rdd2)
```

O comando `.join()`por si só executa o conceito do `innerJoin`. Existem também os demais tipos: `fullOuterJoin`, `leftOuterJoin`, `rightOuterJoin` e `cartesian`.

### Veja mais
É possível encontrar uma lista de todas as transformações disponíveis para uso nos RDDs na API do Spark: http://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

# Exercícios
Para iniciar os exercícios, execute o paragrafo abaixo para que o nosso dataset words esteja disponível para ser utilizado:

In [33]:
text = "At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures. A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only added to, such as counters and sums"
words = text.split(" ")


In [57]:
rdd = sc.parallelize(words, 4)
rdd.setName("palavras")

palavras ParallelCollectionRDD[26] at readRDDFromFile at PythonRDD.scala:262

Para auxiliar na visualização dos resultados, execute o paragrafo abaixo para definir a função display_df():



In [71]:
rdd \
    .map(lambda x: (x,)) \
    .toDF() \
    #.show()

def display_rdd(rdd, n=20):
    return rdd.toDF().limit(n).toPandas()
display_rdd

<function __main__.display_rdd>

1) Retorne a lista de palavras distintas:

In [119]:
rdd2 = rdd.distinct()\
    .map(lambda x: (x,))\

rdd2.take(5)

display_rdd(rdd2)

,_1
0,At
1,of
2,operations
3,provides
4,in
5,are
6,other
7,Hadoop-supported
8,an
9,may


2) Implemente um contador de palavras em lower case:

In [120]:
rdd3 = rdd.map(lambda x: (x.lower(),1))\
  .reduceByKey(lambda a, b: a+b)\
  .sortBy(lambda x: x[1]* -1)

display_rdd(rdd3)


,_1,_2
0,a,12
1,in,9
2,the,8
3,of,6
4,spark,6
5,and,5
6,be,5
7,to,5
8,that,4
9,parallel,4


3)  Realize um inner join do rdd obtido no exercício anterior com o array abaixo utilizando como chave, a palavra em si:

In [121]:
weights_words = [
    ("spark", 1000),
    ("parallel", 500),
    ("function", 300),
    ("driver", 400)
]
weight_rdd = sc.parallelize(weights_words)

rdd4 = rdd3.join(weight_rdd) \
  .map(lambda x: {"word": x[0], "count": x[1][0], "weigth": x[1][1]})

display_rdd(rdd4)

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:401: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


,count,weigth,word
0,4,500,parallel
1,3,400,driver
2,3,300,function
3,6,1000,spark


In [108]:
## solucao 1 rdd2 = rdd.distinct()\
  .map(lambda x: (x,))
rdd2.take(10)
#display_rdd(rdd2)

IndentationError: ignored

In [122]:
#PairRDD = (K,V)
rdd3 = rdd.map(lambda x: (x.lower(),1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1]*-1) # opcional: ordenação
 
display_rdd(rdd3)

,_1,_2
0,a,12
1,in,9
2,the,8
3,of,6
4,spark,6
5,and,5
6,be,5
7,to,5
8,that,4
9,parallel,4


In [123]:
weights_words = [
    ("spark", 1000),
    ("parallel", 500),
    ("function", 300),
    ("driver", 400)
]
 
weight_rdd = sc.parallelize(weights_words)
 
rdd4 = rdd3.join(weight_rdd) \
    .map(lambda x: {"word": x[0], "count": x[1][0], "weight": x[1][1]})
 
display_rdd(rdd4)

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:401: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


,count,weight,word
0,4,500,parallel
1,3,400,driver
2,3,300,function
3,6,1000,spark
